In [12]:
import cupy
import dask.array as da
from time import *
from time import perf_counter

# generate chunked dask arrays of mamy numpy random arrays
rs = da.random.RandomState()

s = 40_000

x = rs.normal(10, 1, size=(s, s), chunks=(10_000, 10_000))
y = rs.normal(10, 1, size=(s, s), chunks=(10_000, 10_000))

print( f"{x.nbytes / 1e9:_} GB" )  # 2 TB

12.8 GB


1203.479483156


In [2]:
%%time 

from IPython.display import clear_output
clear_output()

import numpy
import cupy 
import dask.array as da
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from time import *
from time import perf_counter

devices = [ "cpu", "multi-cpu", "gpu", "multi-gpu" ]

for device in devices : 
    rs = None

    scheduler = "single-threaded" 

    np = None 

    if device.__contains__( "multi" ) :
        scheduler = "threads"
    pass 

    if device.__contains__( "cpu" ):
        print( f"device is {device}")
        np = numpy
    elif device.__contains__( "gpu" ) :
        print( f"device is {device}")
        np = cupy
    pass

    then = perf_counter()

    s = 9_000

    print( f"device = {device}, size = {s:_}, scheduler = {scheduler}" )

    t = np.random.rand( s*s ) + 1j*np.random.rand( s*s )
    t = t.reshape( s, s )
    x = da.from_array( t, chunks=(2**14, 2*14) )
    
    t = np.random.rand( s*s ) + 1j*np.random.rand( s*s )
    t = t.reshape( s, s )
    y = da.from_array( t, chunks=(2**14, 2*14) )    

    z = x*y
    z = z.compute(scheduler=scheduler)

    arrays = [ x, y, z ]

    bytes = 0 

    for array in arrays :
        bytes += array.nbytes
    pass

    print( f"bytes = {bytes/1e9:.2f} Gb" )

    elapsed = perf_counter() - then

    print( f"elapsed = {elapsed:.2f} seconds" )
    print() 

    del t, x, y, z 
pass

device is cpu
device = cpu, size = 90_000, scheduler = single-threaded


In [ ]:

s1 = (x + 1)[::2, ::2].sum().compute(scheduler='single-threaded')
print("s1 = ", s1 )